# Introducing Sources and Samples

In this tutorial, I will explain the basic concepts around which XGA is designed, and why its been set up this way. There won't be much detail on how to use the module for any kind of analysis, but by the end you should have an understanding of how to get started defining sources, then in the next tutorials we can actually start to analyse them.

In [2]:
from astropy.units import Quantity, pix, deg
from astropy.cosmology import Planck15
import numpy as np

# Here we import various types of sample class from XGA
from xga.sources import BaseSource, NullSource, ExtendedSource, PointSource, GalaxyCluster

## What are sources?

XGA revolves around 'source' objects, which are representative of X-ray sources in real life. These are at the heart of any analysis performed with XGA, and just as there are different types of objects that emit X-rays, there are different types of source object built into XGA. We make distinctions between the different types of source due to the different information they can require for their analysis (clusters need overdensity radii for instance, whereas that isn't a useful concept for an AGN).


At their most basic, all that is required to define a source is a position on the sky. The first time XGA is run on a new system, it makes an 'observation census' of the data that you have pointed it to, and finds what observations are available and what their pointing coordinates are; when a new source is defined XGA searches through the census to check whether there are data available for the given coordinates, and if there are relevant observations then they are 'associated' with the source object.

This approach means that the user doesn't have to directly deal with data if they don't want to, XGA will fetch all available data by itself. When it comes to actually analysing and measuring quantities from the source, all the data will be used, not just single observations.

As we ask you to supply region files to XGA in the configuration file, the module can be aware of where other detected sources are in the data its chosen, that allows it to define any 'interloper' sources that have to be excluded from spectrum generation and photometric analysis.


## What types of source are there?

* **BaseSource** - The superclass for all the other source classes, there are very few circumstances where this class should be initialised by users. BaseSource only needs an RA and Dec to be initialised, and if a name is not supplied by the user then one will be generated from those coordinates.


* **NullSource** - This class of source is an exception to XGA's design philosophy that an XGA source represents a real X-ray emitting object. By default NullSource associated every available ObsID with itself (though you may specify which ObsIDs to associate with it), and as such shouldn't be used for astrophysical analysis. This class of source should only be used for bulk generation of products such as images and exposure maps.


* **ExtendedSource** - This is a general class for extended X-ray sources, it is also the superclass of the GalaxyCluster class. XGA will attempt to find a matching extended source from the supplied region files, and if it does then that region will be used for any analysis. The user may also supply a custom circular region in which to analyse the object. Unless it is told not to, XGA will also attempt to find the X-ray peak of this extended object.


* **GalaxyCluster** - As you may have guessed, this class is specifically for Galaxy Clusters, and is a subclass of ExtendedSource. Defining an instance of this class **requires** a redshift to be passed, as well as **at least** one overdensity radius (R$_{200}$, R$_{500}$, and R$_{2500}$ are supported). Also supports passing weak lensing mass and richness values, for use in multi-wavelength analyses. 


* **PointSource** - Similar to the ExtendedSource class in that this is a superclass for more specific point source classes. **This class is implemented and functional, but currently has minimal point source specific abilities.**

If you would like a more specific class implemented for the type of object you're working on, please get in contact with me and I will see what I can do.

## What are samples?

Just as in an a standard analysis, an XGA sample is a group of the same type of object that we wish to analyse as a population; for instance you might want to analyse multiple Galaxy Clusters and derive a scaling relation from them.

There is a secondary benefit to using a sample object rather than multiple Source objects, a sample can be passed into any function that will accept a source, and the function will perform its job on every source in it. This not only makes writing your code easier and cleaner, but can also be more efficient when running SAS and XSPEC, as XGA will run any such jobs in parallel, rather than you having to run them sequentially in a loop for instance.

When a sample is declared (other than the BaseSample class), it will make sure that images and exposure maps for all associated observations are generated for all constinuent sources exist, and if they don't then they will be generated.

## What types of sample are there?

These mostly mirror the types of source that are present in XGA, **though some sample types have not yet been implemented**. Specific sample types can have properties or methods unique to that type of astrophysical object.

* **BaseSample** - The superclass for all the other sample classes, there are very few circumstances where this class should be initialised by users. All a BaseSample requires to be instantiated are two numpy arrays, containing RA and Dec values. Arrays of names and redshifts may also be supplied (though names supplied to **any** sample must be unique).


* **PointSample** - For a population of some generic type of point source. Again only RA and Dec values have to be supplied, though redshift information can be provided if available. As this is a general point source class, no methods to generate scaling relations have been provided.


* **ClusterSample** - For a population of Galaxy Clusters. Just as with the GalaxyCluster source class, here we **require** that redshift and overdensity radius information be provided on declaration. Many convenient features have been added to this sample class, for instance you can retrieve temperatures of all clusters in the sample (if measured) using a ClusterSample method. You can also easily generate common scaling relations by calling methods of the ClusterSample class, using several different fitting methods.

## Defining your first source

Here I demonstrate just how simple it is to define a PointSource object, all I've done is to supply the RA and Dec of Castor, a famous sextuple star system that emits in X-ray. I will note that all coordinates must be passed as decimal degrees; they say that you should be the change you want to see in the world, and I want to see an end to sexagesimal coordinates.

PointSource also accepts various other keyword arguments that you may wish to change from defaults, please see :py:meth:`xga.sources.general.PointSource` for a full list.

In [7]:
demo_src = PointSource(113.65833, 31.87083, name='Castor')

Generating products of type(s) ccf: 100%|██████████| 3/3 [00:25<00:00,  8.36s/it]


We can see from the progress bar above that XGA has detected that no appropriate XMM calibration files were available for the data associated with Castor, and so it used its SAS interface to automatically create the necessary files. It will also have automatically generated combined images and exposure maps, using all available data.

We can now use the info() method to see a summary of the information we have about this particular source. You will notice that XGA has used the input coordinates to find an nH value (using a HEASoft tool). You may also notice that a custom region radius of 0.01 degrees has been used to calculate a SNR value, this is the default region radius for the PointSource class, and may be changed using a keyword argument when the PointSource is defined.

In [16]:
demo_src.info()


-----------------------------------------------------
Source Name - Castor
User Coordinates - (113.65833, 31.87083) degrees
X-ray Peak - (113.65833, 31.87083) degrees
nH - 0.0446 1e+22 / cm2
XMM ObsIDs - 3
PN Observations - 3
MOS1 Observations - 3
MOS2 Observations - 3
On-Axis - 3
With regions - 3
Total regions - 200
Obs with one match - 3
Obs with >1 matches - 0
Images associated - 18
Exposure maps associated - 18
Combined Ratemaps associated - 1
Spectra associated - 0
Custom Region Radius - 0.01 deg
Custom Region SNR - 14.96
-----------------------------------------------------

